In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
import pickle
from sklearn.model_selection import train_test_split

In [ ]:
#Load normalized data


with open('/content/drive/MyDrive/train_texts', 'rb') as f:
  train_texts = pickle.load(f)

with open('/content/drive/MyDrive/train_labels', 'rb') as f:
  train_labels = pickle.load(f)

with open('/content/drive/MyDrive/val_texts', 'rb') as f:
  val_texts = pickle.load(f)

with open('/content/drive/MyDrive/val_labels', 'rb') as f:
  val_labels = pickle.load(f)


In [ ]:
#Define parameters

vocab_size = 50000
oov_tok = "<OOV>"


In [ ]:
#Load tokenizer

with open('/content/drive/MyDrive/tokenizer50k.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)

In [ ]:
'''
word_index = tokenizer.word_index
vocab_size=len(word_index)
vocab_size
'''

789951

In [ ]:
'''
word_index = tokenizer.word_index
vocab_size=len(word_index)
vocab_size+=1
vocab_size
[list(word_index)[-1]]
'''

789952

In [ ]:
[list(tokenizer.word_index)[0]]

['<OOV>']

In [ ]:
tokenizer.texts_to_sequences(['omkarsawant'])

[[1]]

In [ ]:
#Tokenize train and val sentences

train_texts = tokenizer.texts_to_sequences(train_texts)
val_texts = tokenizer.texts_to_sequences(val_texts)

In [ ]:
#Define parameters

trunc_type='post'
max_length = max(len(train_ex) for train_ex in train_texts)
max_length

257

In [ ]:
#Padd train and val sentences to get consistent length

train_texts = pad_sequences(train_texts, maxlen=max_length, truncating=trunc_type)
val_texts = pad_sequences(val_texts, maxlen=max_length, truncating=trunc_type)

In [ ]:
#Define parameters

embedding_dim = 50
print(max_length,vocab_size, embedding_dim)


#Define model

lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

lstm_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
lstm_model.summary()


257 50000 50
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 257, 50)           2500000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 257, 50)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,591,777
Trainable params: 2,591,777
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Fit model

history = lstm_model.fit(train_texts, 
                    train_labels, 
                    batch_size=128,
                    epochs=1,
                    validation_data=(val_texts, val_labels))

22500/22500 [==============================] - 17999s 800ms/step - loss: 0.1716 - accuracy: 0.9348 - val_loss: 0.1353 - val_accuracy: 0.9500


In [ ]:
#Save model to drive

lstm_model.save('/content/drive/MyDrive/lstm_model.hdf5')

In [ ]:
#Test a user defined sentence

sample = ['vary gud']
sample = tokenizer.texts_to_sequences(sample)
sample = pad_sequences(sample, maxlen=max_length, truncating=trunc_type)

lstm_model.predict(sample)

array([[0.62229043]], dtype=float32)

In [ ]:
#Evaluate on test data

with open('/content/drive/MyDrive/test_texts', 'rb') as f:
  test_texts = pickle.load(f)

with open('/content/drive/MyDrive/test_labels', 'rb') as f:
  test_labels = pickle.load(f)

In [ ]:
#Tokenize test sentences

test_texts = tokenizer.texts_to_sequences(test_texts)

In [ ]:
#Padd test sentences to get consistent length

test_texts = pad_sequences(test_texts, maxlen=max_length, truncating=trunc_type)

In [ ]:
#Evaluate

lstm_model.evaluate(test_texts, test_labels, batch_size=128)

3125/3125 [==============================] - 753s 241ms/step - loss: 0.1372 - accuracy: 0.9493


[0.1372213065624237, 0.9493324756622314]

In [ ]:
test_predictions = lstm_model.predict(test_texts)

In [ ]:
test_predictions = [x[0] for x in test_predictions]

In [ ]:
test_predictions = [0 if result<0.5 else 1 for result in test_predictions ]
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, test_predictions)
print(cm)

[[189737  10263]
 [ 10004 189996]]


In [ ]:
from sklearn.metrics import precision_score
precision_score(test_labels, test_predictions, average='binary')

0.9487513669797613

In [ ]:
from sklearn.metrics import recall_score
recall_score(test_labels, test_predictions, average='binary')

0.94998

In [ ]:
from sklearn.metrics import f1_score
f1_score(test_labels, test_predictions)

0.9493652859773296